In [1]:
import cv2
import numpy as np
import urllib.request
import validators
import matplotlib.pyplot as plt


In [2]:
from tritonclient.http import InferenceServerClient

In [22]:
import cv2
import numpy as np
import urllib.request
import validators
import matplotlib.pyplot as plt
from tritonclient.http import InferenceServerClient

# Define function to download image if URL provided
def get_image(image_path):
    if validators.url(image_path):
        with urllib.request.urlopen(image_path) as url_response:
            img_array = np.array(bytearray(url_response.read()), dtype=np.uint8)
            image = cv2.imdecode(img_array, -1)
    else:
        image = cv2.imread(image_path)
    return image

# Define function to perform inference
def infer_model(url, image_path, model):
    # Create InferenceServerClient instance
    client = InferenceServerClient(url=url)

    # Download or read image
    image = get_image(image_path)

    # Preprocess image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Send inference request
    inputs = [InferenceServerClient.InferInput("INPUT0", [-1, -1, 3], "UINT8")]
    inputs[0].set_data_from_numpy(image)
    outputs = client.infer(model_name=model, inputs=inputs)

    # Parse outputs
    names = outputs["names"].tobytes().decode('utf-32').split("|")[:-1]
    bboxes = outputs["bboxes"]
    probs = outputs["probs"]

    # Visualization
    fig, ax = plt.subplots(1, figsize=(10, 10))
    ax.imshow(image)
    ax.axis('off')

    for i in range(len(names)):
        if probs[i] > 0.5:
            x1, y1, x2, y2 = bboxes[i]
            w, h = x2 - x1, y2 - y1
            color = np.random.rand(3,)
            rect = plt.Rectangle((x1, y1), w, h, fill=False, edgecolor=color)
            ax.add_patch(rect)
            color = np.append(color, 0.5)
            ax.text(x1, y1, f"{names[i]} {probs[i]:.2f}", color='black', fontsize=10, bbox=dict(facecolor=color, alpha=0.5))

    plt.show()

# User input and server configuration
image_path = "bus.jpg"
url = "202.92.159.241:8008"
model = "Yolov8x"

# Run inference
infer_model(url, image_path, model)


AttributeError: type object 'InferenceServerClient' has no attribute 'Infer'

In [17]:
def infer_model(url="202.92.159.241:8008", 
                image="bus.jpg", 
                model="Yolov8x"):
    with InferenceServerClient(url, model) as client:
        if validators.url(image):
            with urllib.request.urlopen(image) as url_response:
                img_array = np.array(bytearray(url_response.read()), 
                                     dtype=np.uint8)
                image = cv2.imdecode(img_array, -1)
        else:
            image = cv2.imread(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        outputs = client.infer(inputs=image,model_name=model)
        for k, v in outputs.items():
            if k == "names":
                names = v.tobytes().decode('utf-32').split("|")
                names = names[:-1]
            elif k == "bboxes":
                bboxes = v
            elif k == "probs":
                probs = v
        
        # Plotting
        fig, ax = plt.subplots(1, figsize=(10, 10))
        ax.imshow(image)
        ax.axis('off')
        for i in range(len(names)):
            if probs[i] > 0.5:
                x1, y1, x2, y2 = bboxes[i]
                w = x2 - x1
                h = y2 - y1
                # make the color random
                color = np.random.rand(3,)
                rect = plt.Rectangle((x1, y1), w, h, fill=False, color=color)
                ax.add_patch(rect)
                # use the color and add transparency of 0.5
                color = np.append(color, 0.5)
                # add text with white background
                ax.text(x1, y1, f"{names[i]} {probs[i]:.2f}", color='black', fontsize=10, bbox=dict(facecolor=color, alpha=0.5))
                
                
                

In [18]:
infer_model(image=image_path)

AttributeError: 'numpy.ndarray' object has no attribute '_get_tensor'